In [1]:
import tensorflow as tf

In [1]:
import os
from tensorflow.keras.preprocessing import image 
import numpy as np



In [ ]:
import cv2
import os
import numpy as np

# Input paths
input_base_path = r"C:\Users\creat\Desktop\semesters\7th semester\deepfake_fyp1\BothDatasets\combined_dataset"
real_videos_path = os.path.join(input_base_path, "real")
fake_videos_path = os.path.join(input_base_path, "fake")

# Output paths
output_base_path = r"C:\Users\creat\Desktop\semesters\7th semester\deepfake_fyp1\BothDatasets\FramesExtracted"
real_output_path = os.path.join(output_base_path, "RealFrames")
fake_output_path = os.path.join(output_base_path, "FakeFrames")

# Create output directories if they don't exist
os.makedirs(real_output_path, exist_ok=True)
os.makedirs(fake_output_path, exist_ok=True)

def extract_frames(video_path, output_folder, video_name, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < num_frames:
        print(f"Skipping {video_name}: Not enough frames.")
        cap.release()
        return

    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    saved = 0
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame_filename = f"{video_name}_frame{saved+1:02d}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            saved += 1
    cap.release()

# Process real videos
print("Processing real videos...")
for video_file in os.listdir(real_videos_path):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(real_videos_path, video_file)
        video_name = os.path.splitext(video_file)[0]
        extract_frames(video_path, real_output_path, video_name)

# Process fake videos, but only 590 of them
print("Processing fake videos (limit: 1090 videos)...")
fake_video_count = 0
fake_video_limit = 1090

for video_file in os.listdir(fake_videos_path):
    if fake_video_count >= fake_video_limit:
        break
    if video_file.endswith(".mp4"):
        video_path = os.path.join(fake_videos_path, video_file)
        video_name = os.path.splitext(video_file)[0]
        extract_frames(video_path, fake_output_path, video_name)
        fake_video_count += 1

print("Done extracting frames.")


In [5]:
import os
import cv2

import os
from PIL import Image
import imagehash
import cv2

# Path to the folder containing frames
frames_directory = r"C:\Users\creat\Desktop\semesters\7th semester\deepfake_fyp1\notebooks\frames"

# Load Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
import numpy as np


def FrameCapture(video_path, output_dir="frames", frame_skip=20, min_face_size=60):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    os.makedirs(output_dir, exist_ok=True)

    frame_count = 0
    saved_faces = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=6,
                minSize=(min_face_size, min_face_size)
            )

            for i, (x, y, w, h) in enumerate(faces):
                aspect_ratio = w / float(h)
                if aspect_ratio < 0.75 or aspect_ratio > 1.33:
                    continue

                face = frame[y:y+h, x:x+w]
                brightness = face.mean()
                if brightness < 40:
                    continue

                filename = f"{output_dir}/face_{frame_count}_{i}.jpg"
                cv2.imwrite(filename, face)

                print(f"[✓] Saved {filename}")
                saved_faces += 1

        frame_count += 1

    cap.release()
    print(f"\nDone: {saved_faces} face crops saved.")

def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))

def process_file(path, output_dir="frames", min_face_size=60):
    # Clear existing frames
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            file_path = os.path.join(output_dir, f)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print(f"[✱] Cleared existing files in '{output_dir}'")

    os.makedirs(output_dir, exist_ok=True)
    if is_image_file(path):
        img = cv2.imread(path)
        if img is None:
            print("Error: Cannot read image.")
            return

        os.makedirs(output_dir, exist_ok=True)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=6,
            minSize=(min_face_size, min_face_size)
        )

        total_saved = 0
        for i, (x, y, w, h) in enumerate(faces):
            aspect_ratio = w / float(h)
            if aspect_ratio < 0.75 or aspect_ratio > 1.33:
                continue

            face = img[y:y+h, x:x+w]
            brightness = face.mean()
            if brightness < 40:
                continue

            # Save 30 copies of the face
            for j in range(30):
                filename = f"{output_dir}/image_face_{i}_copy_{j}.jpg"
                cv2.imwrite(filename, face)

                print(f"[✓] Saved {filename}")
                total_saved += 1

        print(f"[→] Detected {len(faces)} valid face(s)")
        print(f"[→] Saved {total_saved} face crops (30 per face)")
    else:
        FrameCapture(path)


def remove_non_face_and_duplicate_frames(directory):
    # Set to store unique hashes of images
    seen_hashes = set()

    # List all files in the directory
    files = os.listdir(directory)

    # Iterate over all files
    for file in files:
        file_path = os.path.join(directory, file)

        # Only consider image files (you can extend this list if needed)
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            try:
                # Open image and compute hash
                image = Image.open(file_path)
                image_hash = imagehash.average_hash(image)

                # Detect faces in the image
                img_cv = cv2.imread(file_path)
                gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                # If no faces are detected, remove the image
                if len(faces) == 0:
                    print(f"No faces detected, removing: {file}")
                    os.remove(file_path)
                else:
                    # Check if the image hash is already in the set (duplicate)
                    if image_hash in seen_hashes:
                        print(f"Duplicate found and removing: {file}")
                        os.remove(file_path)
                    else:
                        # Otherwise, add the hash to the set (unique image with face)
                        seen_hashes.add(image_hash)

            except Exception as e:
                print(f"Error processing file {file}: {e}")




# Example usage
process_file(r"C:\Users\creat\Desktop\Testing videos\interview.mp4")
remove_non_face_and_duplicate_frames(frames_directory)



[✱] Cleared existing files in 'frames'
[✓] Saved frames/face_80_0.jpg
[✓] Saved frames/face_80_1.jpg
[✓] Saved frames/face_100_0.jpg
[✓] Saved frames/face_100_1.jpg
[✓] Saved frames/face_120_0.jpg
[✓] Saved frames/face_140_0.jpg
[✓] Saved frames/face_140_1.jpg
[✓] Saved frames/face_160_0.jpg
[✓] Saved frames/face_160_1.jpg
[✓] Saved frames/face_180_0.jpg
[✓] Saved frames/face_180_1.jpg
[✓] Saved frames/face_200_0.jpg
[✓] Saved frames/face_200_1.jpg
[✓] Saved frames/face_200_2.jpg
[✓] Saved frames/face_220_0.jpg
[✓] Saved frames/face_240_0.jpg
[✓] Saved frames/face_240_1.jpg
[✓] Saved frames/face_260_0.jpg
[✓] Saved frames/face_300_0.jpg
[✓] Saved frames/face_300_1.jpg
[✓] Saved frames/face_300_2.jpg
[✓] Saved frames/face_320_0.jpg
[✓] Saved frames/face_320_1.jpg
[✓] Saved frames/face_320_2.jpg
[✓] Saved frames/face_340_0.jpg
[✓] Saved frames/face_360_0.jpg
[✓] Saved frames/face_360_1.jpg
[✓] Saved frames/face_380_0.jpg
[✓] Saved frames/face_400_0.jpg
[✓] Saved frames/face_420_0.jpg
[✓]

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model


# Load your saved Xception-based model
loaded_model = load_model(r"C:\Users\creat\Desktop\semesters\7th semester\deepfake_fyp1\models\best models\fine_tuned_model.h5")

In [6]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import load_model

def evaluate_video_sequence(directory, loaded_model, input_size=(224, 224)):
    frame_files = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    
    if not frame_files:
        print("No image frames found in the directory.")
        return

    frames = []
    for filename in frame_files:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Couldn't read {filename}, skipping.")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, input_size)
        img = preprocess_input(img.astype(np.float32))
        frames.append(img)

    if not frames:
        print("No valid frames to process.")
        return

    input_batch = np.array(frames)

    # Predict on each frame
    predictions = loaded_model.predict(input_batch, verbose=0)  # shape: (num_frames, 2)
    
    # Print confidence for each frame
    print("Per-frame predictions (Fake class confidence):")
    for i, pred in enumerate(predictions):
        print(f"  Frame {i + 1}: {pred[1]:.4f}")

    # Take average prediction for class 1 (e.g., "Fake")
    confidence = np.mean(predictions[:, 1])
    label = "Fake" if confidence >= 0.5 else "Real"
    print(f"\nFinal Prediction: {label} (Average Confidence: {confidence:.4f})")

# Usage example (ensure your model is loaded):
# loaded_model = load_model("deepfake_model.h5")
evaluate_video_sequence("frames", loaded_model)

Per-frame predictions (Fake class confidence):
  Frame 1: 0.5499
  Frame 2: 0.5206
  Frame 3: 0.9902
  Frame 4: 0.9489
  Frame 5: 0.3430
  Frame 6: 0.9839
  Frame 7: 0.3598
  Frame 8: 0.9796
  Frame 9: 0.8406
  Frame 10: 0.4805
  Frame 11: 0.4464
  Frame 12: 0.9629
  Frame 13: 0.8704
  Frame 14: 0.8946
  Frame 15: 0.8319
  Frame 16: 0.8445
  Frame 17: 0.8209
  Frame 18: 0.8679
  Frame 19: 0.9470
  Frame 20: 0.9821
  Frame 21: 0.9425
  Frame 22: 0.9329
  Frame 23: 0.9493
  Frame 24: 0.8749
  Frame 25: 0.8218
  Frame 26: 0.9806
  Frame 27: 0.7248
  Frame 28: 0.9019
  Frame 29: 0.9373
  Frame 30: 0.8494
  Frame 31: 0.8664
  Frame 32: 0.8919
  Frame 33: 0.8977
  Frame 34: 0.8587
  Frame 35: 0.9077
  Frame 36: 0.9072
  Frame 37: 0.8723
  Frame 38: 0.9799
  Frame 39: 0.8419
  Frame 40: 0.9052
  Frame 41: 0.7904
  Frame 42: 0.9866
  Frame 43: 0.8477
  Frame 44: 0.9188
  Frame 45: 0.8002
  Frame 46: 0.8375
  Frame 47: 0.8663
  Frame 48: 0.8236
  Frame 49: 0.9193
  Frame 50: 0.9242
  Frame 51: 

In [13]:
import tensorflow as tf
import gc
from tensorflow.python.framework import ops

def clear_gpu_memory():
    # Clear TensorFlow session
    tf.keras.backend.clear_session()
    
    # Collect garbage
    gc.collect()
    
    # Reset the default graph
    ops.reset_default_graph()

    print("GPU memory cleared!")

# Call this function when you need to clear the memory
clear_gpu_memory()


GPU memory cleared!


In [14]:
import tensorflow as tf
import gc

# Reset session and clear memory
tf.keras.backend.clear_session()

# Optional: run garbage collector
gc.collect()

print("✅ TensorFlow GPU memory cleared.")


✅ TensorFlow GPU memory cleared.
